# <font color='green'>Sistema de Recomendação</font>

Este projeto tem como objetivo aplicar um algoritmo de Sistema de Recomendação e calcular o affinity score, a fim de descobrir se um produto deve ou não ser recomendado a um usuário.

Foi utilizado um dataset de exemplo onde cada coluna representa, respectivamente: user, item, rating

In [2]:
spSession = SparkSession.builder.master("local").appName("SR-SparkMLLib").config("spark.some.config.option", "session").getOrCreate()

In [3]:
from pyspark.ml.recommendation import ALS

## Pré-processamento

In [4]:
userItemRDD = sc.textFile("data/user-item.txt")

In [5]:
userItemRDD.collect()

['1001,9001,10',
 '1001,9002,1',
 '1001,9003,9',
 '1002,9001,3',
 '1002,9002,5',
 '1002,9003,1',
 '1002,9004,10',
 '1003,9001,2',
 '1003,9002,6',
 '1003,9003,2',
 '1003,9004,9',
 '1003,9005,10',
 '1003,9006,8',
 '1003,9007,9',
 '1004,9001,9',
 '1004,9002,2',
 '1004,9003,8',
 '1004,9004,3',
 '1004,9010,10',
 '1004,9011,9',
 '1004,9012,8',
 '1005,9001,8',
 '1005,9002,3',
 '1005,9003,7',
 '1005,9004,1',
 '1005,9010,9',
 '1005,9011,10',
 '1005,9012,9',
 '1005,9013,8',
 '1005,9014,1',
 '1005,9015,1',
 '1006,9001,7',
 '1006,9002,4',
 '1006,9003,8',
 '1006,9004,1',
 '1006,9010,7',
 '1006,9011,6',
 '1006,9012,9']

In [13]:
userItemRDD2 = userItemRDD.map(lambda l: l.split(',')).map(lambda l:(int(l[0]), int(l[1]), float(l[2])))
userItemRDD2.take(5)

[(1001, 9001, 10.0),
 (1001, 9002, 1.0),
 (1001, 9003, 9.0),
 (1002, 9001, 3.0),
 (1002, 9002, 5.0)]

In [15]:
# Criando um Dataframe a partir do RDD
userItemDF = spSession.createDataFrame(userItemRDD2, ["user", "item", "rating"])
userItemDF.show()

+----+----+------+
|user|item|rating|
+----+----+------+
|1001|9001|  10.0|
|1001|9002|   1.0|
|1001|9003|   9.0|
|1002|9001|   3.0|
|1002|9002|   5.0|
|1002|9003|   1.0|
|1002|9004|  10.0|
|1003|9001|   2.0|
|1003|9002|   6.0|
|1003|9003|   2.0|
|1003|9004|   9.0|
|1003|9005|  10.0|
|1003|9006|   8.0|
|1003|9007|   9.0|
|1004|9001|   9.0|
|1004|9002|   2.0|
|1004|9003|   8.0|
|1004|9004|   3.0|
|1004|9010|  10.0|
|1004|9011|   9.0|
+----+----+------+
only showing top 20 rows



## Construção do modelo

Para o sistema de recomendação, utilizando Filtragem Colaborativa, será utilizado o algoritmo ALS (Alternating Least Squares) que otimiza a loss function e tem bom desempenho em ambientes paralelizados.